In [1]:
from pathlib import Path
import torch
        
from numpy import load #Chameleon and Squirrel filtered
from torch_geometric.data import Data

from torch_geometric.datasets import Planetoid, Amazon, Actor, WikipediaNetwork, HeterophilousGraphDataset, WikiCS
from ogb.nodeproppred import PygNodePropPredDataset
from tqdm import tqdm


In [2]:
import yaml
from data import Dataset

In [3]:
#load splits and get stats

In [4]:
def get_ds(name, split):
    settings = {"dataset":name, "split":split, "device":"cpu", "datamode":"gra", "ds_make_undir": True, "ds_add_self": False} #default settings
    ds = Dataset(settings)
    return ds
def print_stats(ds):
    n, e = 0, 0
    for rep in range(10):
        ds.set_rep(rep)
        d = ds.get_train()
        n += d.x.shape[0]
        e += d.edge_index.shape[1]
    #print("Nodes:", n/10)
    #print("Edges:", e/10)
    return "$"+str(int(n/10))+"/"+str(e/10)+"$"

In [46]:
import torch_geometric.transforms as T

for ds_name in ["Cora", "Citeseer", "Pubmed"]:
    
    transform = T.Compose([T.ToUndirected()])
    dat = Planetoid(root='dataset/'+ds_name+"/", name=ds_name, transform=transform)[0]
    ds = dat.subgraph(dat.train_mask)
    print(ds_name, "planetoid; only train")
    print("Nodes:", ds.x.shape[0])
    print("Edges:", ds.edge_index.shape[1])
    
    msk = torch.ones_like(dat.train_mask)
    msk[dat.val_mask] = 0
    msk[dat.test_mask] = 0
    ds = dat.subgraph(msk)
    print(ds_name, "planetoid; train + structure (nodes not in val or test)")
    print("Nodes:", ds.x.shape[0])
    print("Edges:", ds.edge_index.shape[1])
    ds = dat
    print(ds_name, "full planetoid (train+val+test+structure/unused)")
    print("Nodes:", ds.x.shape[0])
    print("Edges:", ds.edge_index.shape[1])
    print()



Cora planetoid; only train
Nodes: 140
Edges: 42
Cora planetoid; train + structure (nodes not in val or test)
Nodes: 1208
Edges: 2308
Cora full planetoid (train+val+test+structure/unused)
Nodes: 2708
Edges: 10556

Citeseer planetoid; only train
Nodes: 120
Edges: 16
Citeseer planetoid; train + structure (nodes not in val or test)
Nodes: 1827
Edges: 2642
Citeseer full planetoid (train+val+test+structure/unused)
Nodes: 3327
Edges: 9104

Pubmed planetoid; only train
Nodes: 60
Edges: 0
Pubmed planetoid; train + structure (nodes not in val or test)
Nodes: 18217
Edges: 75472
Pubmed full planetoid (train+val+test+structure/unused)
Nodes: 19717
Edges: 88648



In [19]:
def largest_class_percentage(dss):
    for ds in dss:
        print()
        print(ds)
        dataset = get_ds(ds, "81")
        dataset.set_rep(0)
        y = dataset.get_test().y
        print(y.unique(return_counts=True)[1].max()/y.shape[0])
    

In [20]:
largest_class_percentage(["Cora", "Citeseer", "Pubmed", "Computers", "Photo", "WikiCS", "Actor", "Roman-empire", "Amazon-ratings"])


Cora
tensor(0.3021)

Citeseer
tensor(0.2107)

Pubmed
tensor(0.3994)

Computers
tensor(0.3751)

Photo
tensor(0.2537)

WikiCS
tensor(0.2290)

Actor
tensor(0.2586)

Roman-empire
tensor(0.1396)

Amazon-ratings
tensor(0.3679)


In [36]:
def zero_features(dss):
    for ds in dss:
        print()
        print(ds)
        dataset = get_ds(ds, "81")
        dataset.set_rep(0)
        x = dataset.get_test().x
        print(((x == 0).all(1)).sum())

In [37]:
zero_features(["Cora", "Citeseer", "Pubmed", "Computers", "Photo", "WikiCS", "Actor", "Roman-empire", "Amazon-ratings"])


Cora
tensor(0)

Citeseer
tensor(15)

Pubmed
tensor(0)

Computers
tensor(0)

Photo
tensor(0)

WikiCS
tensor(1)

Actor
tensor(0)

Roman-empire
tensor(62)

Amazon-ratings
tensor(0)


In [61]:
def print_all(dss, splits):
    for ds in dss:
        print()
        print(ds)
        tmp = [ds, "$|V|$/$|E|$"]
        for split in splits:
            dataset = get_ds(ds, split)
            #print({"145":"10", "24":"20", "43":"40", "62":"60", "81":"80"}[split])
            tmp.append(print_stats(dataset))
        tmp[-1] = tmp[-1]+" \\\\ \\hline"
        print(" & ".join(tmp))
    

In [62]:
print_all(["Cora", "Citeseer", "Pubmed", "Computers", "Photo", "Actor", "Roman-empire", "Amazon-ratings"], ["145", "24", "43", "62", "81"])


Cora
Cora & $|V|$/$|E|$ & $271/105.2$ & $542/436.6$ & $1083/1764.0$ & $1625/3902.4$ & $2166/6852.0$ \\ \hline

Citeseer
Citeseer & $|V|$/$|E|$ & $333/84.2$ & $665/351.4$ & $1331/1440.0$ & $1996/3236.2$ & $2662/5730.0$ \\ \hline

Pubmed
Pubmed & $|V|$/$|E|$ & $1972/859.8$ & $3943/3511.8$ & $7887/14201.6$ & $11830/31837.0$ & $15774/56512.6$ \\ \hline

Computers
Computers & $|V|$/$|E|$ & $1375/4798.6$ & $2750/19966.0$ & $5501/78283.6$ & $8251/179890.4$ & $11002/314372.4$ \\ \hline

Photo
Photo & $|V|$/$|E|$ & $765/2326.2$ & $1530/9221.4$ & $3060/37296.0$ & $4590/85105.8$ & $6120/152316.8$ \\ \hline

Actor
Actor & $|V|$/$|E|$ & $760/532.0$ & $1520/2262.5$ & $3040/8870.8$ & $4560/19690.0$ & $6080/34273.1$ \\ \hline

Roman-empire
Roman-empire & $|V|$/$|E|$ & $2266/664.8$ & $4532/2635.6$ & $9065/10574.4$ & $13597/23707.0$ & $18130/42179.8$ \\ \hline

Amazon-ratings
Amazon-ratings & $|V|$/$|E|$ & $2449/1831.4$ & $4898/7420.2$ & $9797/29754.2$ & $14695/67068.4$ & $19594/119215.0$ \\ \hline


In [8]:
from torch_geometric.utils import homophily, degree
    
    

In [9]:
# https://arxiv.org/pdf/2209.06177
def adjusted_homophily(e, y):
    edge_homophily = homophily(e,y,method="edge")
    deg = degree(e[0,:], num_nodes = len(y)) + degree(e[1,:], num_nodes = len(y)) #in+out degree
    lE = 2*e.shape[1]
    Dk = [0]*(y.max()+1)
    for i in range(len(y)):
        Dk[y[i]] += deg[i]
    pk = [(x/lE)**2 for x in Dk]
    adj_homophily = (edge_homophily - sum(pk)) / (1-sum(pk))
    print("edge_homophily", edge_homophily)
    print("adj_homophily", float(adj_homophily))


In [47]:
ds_name ="Cora"
ds = Planetoid(root='dataset/'+ds_name+"/", name=ds_name)[0]
adjusted_homophily(ds.edge_index, ds.y)

edge_homophily 0.8099659085273743
adj_homophily 0.7710854411125183


In [48]:
ds_name ="Citeseer"
ds = Planetoid(root='dataset/'+ds_name+"/", name=ds_name)[0]
adjusted_homophily(ds.edge_index, ds.y)

edge_homophily 0.7355008721351624
adj_homophily 0.6706594228744507


In [49]:
ds_name ="Pubmed"
ds = Planetoid(root='dataset/'+ds_name+"/", name=ds_name)[0]
adjusted_homophily(ds.edge_index, ds.y)

edge_homophily 0.8023869395256042
adj_homophily 0.6860214471817017


In [54]:
ds_name = "Photo"
ds = Amazon(root='dataset/'+ds_name+"/", name=ds_name)[0]
adjusted_homophily(ds.edge_index, ds.y)

edge_homophily 0.8272436261177063
adj_homophily 0.7850189805030823


In [55]:
ds_name = "Computers"
ds = Amazon(root='dataset/'+ds_name+"/", name=ds_name)[0]
adjusted_homophily(ds.edge_index, ds.y)

edge_homophily 0.7772156000137329
adj_homophily 0.6823362708091736


In [10]:
ds_name ="WikiCS"
ds = WikiCS(root='dataset/'+ds_name+"/")[0]
adjusted_homophily(ds.edge_index, ds.y)

edge_homophily 0.6547393202781677
adj_homophily 0.5795594453811646


In [58]:
ds_name = "Actor"
ds = Actor(root="dataset/Actor/")[0]
print(ds.x.shape, ds.edge_index.shape, ds.y.max(), ds.y.shape)
adjusted_homophily(ds.edge_index, ds.y)

torch.Size([7600, 932]) torch.Size([2, 30019]) tensor(4) torch.Size([7600])
edge_homophily 0.21876144409179688
adj_homophily 0.005505717825144529


In [50]:
ds_name = "Roman-empire"
ds = HeterophilousGraphDataset(root='dataset/'+ds_name+"/", name=ds_name)[0]
adjusted_homophily(ds.edge_index, ds.y)

edge_homophily 0.04689160734415054
adj_homophily -0.04675774276256561


In [51]:
ds_name = "Amazon-ratings"
ds = HeterophilousGraphDataset(root='dataset/'+ds_name+"/", name=ds_name)[0]
adjusted_homophily(ds.edge_index, ds.y)

edge_homophily 0.3803761303424835
adj_homophily 0.140230193734169


In [3]:
#create random % splits
def create_split(save_path, dataset, train_p = .6, val_p =.2, test_p =.2, splits = 10):
    assert train_p+val_p+test_p == 1

    train_mask, val_mask, test_mask = [], [], []
    
    for seed in tqdm(range(splits)):
        torch.manual_seed(seed)

        n = dataset.num_nodes
        tv = round(train_p*n)
        vt = round((train_p+val_p)*n)
        
        perm = torch.randperm(n)

        train_ix = perm[:tv]
        val_ix = perm[tv:vt]
        test_ix = perm[vt:]

        train = torch.full_like(dataset.y.squeeze(), False, dtype=torch.bool)
        train[train_ix] = True
        val = torch.full_like(dataset.y.squeeze(), False, dtype=torch.bool)
        val[val_ix] = True
        test = torch.full_like(dataset.y.squeeze(), False, dtype=torch.bool)
        test[test_ix] = True
        train_mask.append(train)
        val_mask.append(val)
        test_mask.append(test)
    dict = {"train":torch.stack(train_mask, 1), "valid":torch.stack(val_mask, 1), "test":torch.stack(test_mask, 1)}
    assert torch.logical_and(dict["train"], dict["valid"]).sum() == 0
    assert torch.logical_and(dict["train"], dict["test"]).sum() == 0
    assert torch.logical_and(dict["valid"], dict["test"]).sum() == 0
    assert torch.logical_or(torch.logical_or(dict["train"], dict["valid"]), dict["test"]).all()
    torch.save(dict, save_path)

In [3]:
p = Path("dataset/ogbn-arxiv")
dataset = PygNodePropPredDataset(name = "ogbn-arxiv", root = str(p))
create_split("dataset/ogbn-arxiv/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/ogbn-arxiv/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/ogbn-arxiv/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/ogbn-arxiv/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/ogbn-arxiv/own_81_splits.pt", dataset[0], .8, .1, .1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 359.40it/s]


In [3]:
p = Path("dataset/Roman-empire/")
dataset = HeterophilousGraphDataset(str(p), name='Roman-empire')
create_split("dataset/Roman-empire/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/Roman-empire/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/Roman-empire/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/Roman-empire/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/Roman-empire/own_81_splits.pt", dataset[0], .8, .1, .1)

Processing...
Done!
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1607.38it/s]


In [4]:
p = Path("dataset/Amazon-ratings/")
dataset = HeterophilousGraphDataset(str(p), name='Amazon-ratings')
create_split("dataset/Amazon-ratings/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/Amazon-ratings/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/Amazon-ratings/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/Amazon-ratings/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/Amazon-ratings/own_81_splits.pt", dataset[0], .8, .1, .1)

Processing...
Done!
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1739.51it/s]


In [3]:
p = Path("dataset/Cora/")
dataset = Planetoid(str(p), name='Cora')
create_split("dataset/Cora/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/Cora/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/Cora/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/Cora/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/Cora/own_81_splits.pt", dataset[0], .8, .1, .1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 5477.02it/s]


In [5]:
p = Path("dataset/Citeseer/")
dataset = Planetoid(str(p), name='Citeseer')
create_split("dataset/Citeseer/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/Citeseer/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/Citeseer/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/Citeseer/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/Citeseer/own_81_splits.pt", dataset[0], .8, .1, .1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 4743.61it/s]


In [7]:
p = Path("dataset/Pubmed/")
dataset = Planetoid(str(p), name='Pubmed')
create_split("dataset/Pubmed/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/Pubmed/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/Pubmed/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/Pubmed/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/Pubmed/own_81_splits.pt", dataset[0], .8, .1, .1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1864.55it/s]


In [10]:
p = Path("dataset/Computers/")
dataset = Amazon(str(p), name='Computers')
create_split("dataset/Computers/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/Computers/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/Computers/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/Computers/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/Computers/own_81_splits.pt", dataset[0], .8, .1, .1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2370.33it/s]


In [11]:
p = Path("dataset/Photo/")
dataset = Amazon(str(p), name='Photo')
create_split("dataset/Photo/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/Photo/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/Photo/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/Photo/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/Photo/own_81_splits.pt", dataset[0], .8, .1, .1)

Processing...
Done!
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 3044.20it/s]


In [4]:
p = Path("dataset/WikiCS/")
dataset = WikiCS(str(p))
create_split("dataset/WikiCS/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/WikiCS/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/WikiCS/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/WikiCS/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/WikiCS/own_81_splits.pt", dataset[0], .8, .1, .1)

/media/data2/nlell/struct/lib/python3.11/site-packages/torch_geometric/datasets/wikics.py:45: UserWarning: The WikiCS dataset now returns an undirected graph by default. Please explicitly specify 'is_undirected=False' to restore the old behavior.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2322.17it/s]


In [14]:
p = Path("dataset/Actor/")
dataset = Actor(str(p))
create_split("dataset/Actor/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/Actor/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/Actor/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/Actor/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/Actor/own_81_splits.pt", dataset[0], .8, .1, .1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1886.18it/s]


In [4]:
p = Path("dataset/Chameleon/")
dataset = WikipediaNetwork(root = str(p), name = "Chameleon")
create_split("dataset/Chameleon/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/Chameleon/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/Chameleon/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/Chameleon/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/Chameleon/own_81_splits.pt", dataset[0], .8, .1, .1)

Processing...
Done!
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 5461.33it/s]


In [10]:
np_data = load('dataset/Chameleon-f/chameleon_filtered.npz')
data = Data(x=torch.from_numpy(np_data["node_features"]), y=torch.from_numpy(np_data["node_labels"]), edge_index=torch.from_numpy(np_data["edges"]))
create_split("dataset/Chameleon-f/own_145_splits.pt", data, .1, .45, .45)
create_split("dataset/Chameleon-f/own_24_splits.pt", data, .2, .4, .4)
create_split("dataset/Chameleon-f/own_43_splits.pt", data, .4, .3, .3)
create_split("dataset/Chameleon-f/own_62_splits.pt", data, .6, .2, .2)
create_split("dataset/Chameleon-f/own_81_splits.pt", data, .8, .1, .1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 5471.31it/s]


In [5]:
p = Path("dataset/Squirrel/")
dataset = WikipediaNetwork(root = str(p), name = "Squirrel")
create_split("dataset/Squirrel/own_145_splits.pt", dataset[0], .1, .45, .45)
create_split("dataset/Squirrel/own_24_splits.pt", dataset[0], .2, .4, .4)
create_split("dataset/Squirrel/own_43_splits.pt", dataset[0], .4, .3, .3)
create_split("dataset/Squirrel/own_62_splits.pt", dataset[0], .6, .2, .2)
create_split("dataset/Squirrel/own_81_splits.pt", dataset[0], .8, .1, .1)

Processing...
Done!
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 3261.77it/s]


In [12]:
np_data = load('dataset/Squirrel-f/squirrel_filtered.npz')
data = Data(x=torch.from_numpy(np_data["node_features"]), y=torch.from_numpy(np_data["node_labels"]), edge_index=torch.from_numpy(np_data["edges"]))
create_split("dataset/Squirrel-f/own_145_splits.pt", data, .1, .45, .45)
create_split("dataset/Squirrel-f/own_24_splits.pt", data, .2, .4, .4)
create_split("dataset/Squirrel-f/own_43_splits.pt", data, .4, .3, .3)
create_split("dataset/Squirrel-f/own_62_splits.pt", data, .6, .2, .2)
create_split("dataset/Squirrel-f/own_81_splits.pt", data, .8, .1, .1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 5183.91it/s]
